In [4]:
import pickle as pkl
import pandas as pd

# Imputer
with open('../model/imputer.pkl','rb') as fichier_pickler:
    imputer = pkl.load(fichier_pickler)


# Classification
with open('../model/modele_classification.pkl','rb') as fichier_pickler:
    model_classif = pkl.load(fichier_pickler)



# Regression
with open('../model/modele_regression_code_1.pkl','rb') as fichier_pickler:
    model_reg_1 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_2.pkl','rb') as fichier_pickler:
    model_reg_2 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_3.pkl','rb') as fichier_pickler:
    model_reg_3 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_4.pkl','rb') as fichier_pickler:
    model_reg_4 = pkl.load(fichier_pickler)



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

colonnes_a_imputer = ["Surface reelle bati","Nombre pieces principales"]


transformateur_medianne = ColumnTransformer(
    transformers=[('imputer',imputer,colonnes_a_imputer)]
)

# Pipeline classif
pipeline_classif = Pipeline([
    ('median_imputer',transformateur_medianne),
    ('classifier',model_classif)
])

# Pipeline Type Local == 1
pipeline_reg_1 = Pipeline([
    ('reg_1',model_reg_1)
])

pipeline_reg_2 = Pipeline([
    ('reg_2',model_reg_2)
])

pipeline_reg_3 = Pipeline([
    ('reg_3',model_reg_3)
])

pipeline_reg_4 = Pipeline([
    ('reg_4',model_reg_4)
])